In [1]:
# coding: utf8
import random
import time
import math
import tkinter
from tkinter import messagebox

SPACE = 0
BLACK = 1
WHITE = -1
BOARD_SQUARE = 8
BOARD_SQUARE_X = BOARD_SQUARE
BOARD_SQUARE_Y = BOARD_SQUARE
BOARD_PX_SIZE = 500
BOARD_CELL_PX_SIZE = BOARD_PX_SIZE / BOARD_SQUARE

In [2]:
class Position:
    def __init__(self):
        self.y = 0
        self.x = 0
  
    def __init__(self, y, x):
        self.y = y
        self.x = x

In [10]:
class Board:
    DIR = [[-1, -1], [-1, 0], [-1, 1],
           [ 0, -1],          [ 0, 1],
           [ 1, -1], [ 1, 0], [ 1, 1]]
    
    def __init__(self):
        self.turn = BLACK
        self.move_num = 1
        self.board = [[SPACE for i in range(8)] for j in range(8)]
        #  ---- board
        # [[0, 0, 0, 0, 0, 0, 0, 0],
        #  [0, 0, 0, 0, 0, 0, 0, 0],
        #  [0, 0, 0, 0, 0, 0, 0, 0],
        #  [0, 0, 0, 0, 0, 0, 0, 0],
        #  [0, 0, 0, 0, 0, 0, 0, 0],
        #  [0, 0, 0, 0, 0, 0, 0, 0],
        #  [0, 0, 0, 0, 0, 0, 0, 0],
        #  [0, 0, 0, 0, 0, 0, 0, 0]]

    # Initialize the board. / 盤面を初期化
    def init_board(self):
        self.board = [[SPACE for i in range(BOARD_SQUARE_X)] for j in range(BOARD_SQUARE_Y)]
        self.board[3][3] = WHITE
        self.board[3][4] = BLACK
        self.board[4][3] = BLACK
        self.board[4][4] = WHITE
        self.turn = BLACK  # turn / 手番
        self.move_num = 1  # number of moves in attack./ 手数
  
    # Count black and white pieces. / 白と黒のピースの数を数える
    def get_discs(self):
        black_discs = 0
        white_discs = 0
        for y in range(BOARD_SQUARE_Y):
            for x in range(c):
                disc = self.board[y][x]
            if disc == BLACK:
                black_discs += 1
            elif disc == WHITE:
                white_discs += 1
        return (black_discs, white_discs)

    # Can place a new piece or not? / ピースを配置できるかどうか
    def is_movable(self, position):
        # Is the position ocupied or opened? / 配置位置が空いているか
        if self.board[position.y][position.x] != SPACE:
            return False
    
        # Can flipp white/black to black/white?
        # ピースを裏返すことが出来るか
        for dir in Board.DIR:
            # Get coordinate around the original position.
            # 指定位置の周りの座標を取得
            y = position.y + dir[0]
            x = position.x + dir[1]
            
            if y >= 0 and x >= 0 and \
                y < BOARD_SQUARE_Y and x < BOARD_SQUARE_X and \
                self.board[y][x] == -self.turn: # Is it an opponent peice? / 相手のピースか
                y += dir[0]
                x += dir[1]
                # Move strait as long as the opponent's piece continues.
                # 相手のピースが続いている限り直線的に移動
                while y >= 0 and x >= 0 and y < 8 and x < 8 and self.board[y][x] == -self.turn:
                    y += dir[0]
                    x += dir[1]
                # After moving in a straight line, check if my piece is at the end point.
                # 直線的に移動した後、終点に自分のピースがあるかをチェック
                if y >= 0 and x >= 0 and y < 8 and x < 8 and self.board[y][x] == self.turn:
                    return True
      
        return False
  
    # Get a list of squares which can put peice in. / ピースを置けるマスのリストを取得 
    def get_move_list(self):
        move_list = []
        for y in range(8):
            for x in range(8):
                if self.board[y][x] == SPACE:
                    position = Position(y, x)
                    if self.is_movable(position):
                        move_list.append(position)
        return move_list
#-------------------------------------------
    # 局面を進める
    def move(self, position):
        # 石を打つ
        self.board[position.y][position.x] = self.turn
        
        # 石をひっくり返す
        # 各方向に石をひっくり返せるか調べる
        for dir in Board.DIR:
            y = position.y + dir[0]
            x = position.x + dir[1]
            if y >= 0 and x >= 0 and y < 8 and x < 8 and self.board[y][x] == -self.turn:
                # 隣が相手の石
                y += dir[0]
                x += dir[1]
                while y >= 0 and x >= 0 and y < 8 and x < 8 and self.board[y][x] == -self.turn:
                    y += dir[0]
                    x += dir[1]
                if y >= 0 and x >= 0 and y < 8 and x < 8 and self.board[y][x] == self.turn:
                    # この方向は返せる
                    # 1マス戻る
                    y -= dir[0]
                    x -= dir[1]
                    # 戻りながら返す
                    while y >= 0 and x >= 0 and y < 8 and x < 8 and self.board[y][x]==-self.turn:
                        self.board[y][x] = self.turn
                        y -= dir[0]
                        x -= dir[1]
    
        self.turn = -self.turn  # 手番を変更
        self.move_num += 1    # 手数を増やす

    # パスする
    def move_pass(self):
        self.turn = -self.turn  # パス
  
    # 対局終了の判定
    def is_game_end(self):
        # 60手に達した時
        if self.move_num == 61:
            return True

        # 黒白どちらかの石数が0になった時
        # 黒白の石数をタプルで取得
        (black_discs, white_discs) = self.get_discs()
        if black_discs == 0 or white_discs == 0:
            return True

        # 黒白どちらも手がない場合
        move_list1 = self.get_move_list()
        if len(move_list1) == 0:
            self.move_pass()  # パスして相手番にする
            move_list2 = self.get_move_list()
            self.move_pass()  # パスして戻す
            if len(move_list2) == 0:
                return True

        return False